In [3]:
print(5//2)

2


In [ ]:
#from google.colab import drive
import pandas as pd
import numpy as np
import librosa
import librosa.display
import json
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.image import resize
from sklearn.model_selection import train_test_split
#from tensorflow.keras.models import Sequential, Model
#from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input, BatchNormalization, Concatenate, GRU, TimeDistributed
import concurrent.futures
import time
from tensorflow.keras.metrics import AUC
import h5py
from scipy.signal import butter, filtfilt
from scipy.ndimage import gaussian_filter1d
import wave
import scipy.io.wavfile as wavf


In [3]:
meta_data = pd.read_csv('audio_metadata - filtered.csv', sep=',')
meta_data.drop(columns=['group', 'category', 'sub_category'], inplace=True)

In [4]:
df_id = np.array(meta_data['id'].astype(int)).flatten()

wavs = []
srs = []

def load_sample(f, sample_id):
    group = f["audio_data"][str(sample_id)]
    waveform = np.array(group["waveform"])
    sample_rate = np.array(group["sample_rate"])
    return waveform, sample_rate

with h5py.File('filtered_audio_data.h5', "r") as f:
    for i in df_id:
        waveform, sr = load_sample(f, i)  # load sample with id=42
        wavs.append(waveform)
        srs.append(sr)

In [5]:
samples = pd.DataFrame({'id': df_id, 'waveform': wavs, 'sample_rate': srs})
samples['beginning_time'] = samples.apply(lambda row: len(row['waveform']) / row['sample_rate'], axis=1)

In [6]:
def highpass_filter(y, sr, cutoff=10.0, order=2):
    nyquist = 0.5 * sr
    normal_cutoff = cutoff / nyquist
    b, a = butter(order, normal_cutoff, btype='high', analog=False)
    return filtfilt(b, a, y)

def lowpass_filter(y, sr, cutoff=150.0, order=2):
    nyquist = 0.5 * sr
    normal_cutoff = cutoff / nyquist
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return filtfilt(b, a, y)

def bandpass_filter(y, sr, lowcut=10.0, highcut=150.0, order=2):
    nyquist = 0.5 * sr
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band', analog=False)
    return filtfilt(b, a, y)

def preprocess_audio(y, sr, top_db=120):
    y = librosa.resample(y, orig_sr=sr, target_sr=44100)
    y = highpass_filter(y, sr)
    y_rev = y[::-1]
    y_trimmed, _ = librosa.effects.trim(y_rev, top_db=top_db)
    y_reversed_back = y_trimmed[::-1]
    return y_reversed_back

In [ ]:
samples['waveform'] = samples.apply(lambda row: preprocess_audio(row['waveform'], row['sample_rate']), axis=1)
samples['sample_rate'] = 44100

In [367]:
sample = samples.iloc[i]['waveform'].astype(np.float32)

sr = 44100
bpm = np.random.randint(70, 151)
print(bpm)
num_of_beats = 2 ** np.random.randint(2, 5)
num_of_hits = np.random.randint(2, 4 * num_of_beats)

beat_time = 60 / bpm  # seconds per beat
loop_duration = beat_time * num_of_beats
loop_samples = int(np.ceil(loop_duration * sr))
new_loop = np.zeros(loop_samples, dtype=np.float32)

# 16th note resolution
hits = np.zeros(num_of_beats * 4)
hits[:num_of_hits] = 1
np.random.shuffle(hits)

print(hits)

for i, hit in enumerate(hits):
    if hit == 1:
        start_idx = int(i * (sr * beat_time / 4))
        end_idx = start_idx + len(sample)
        if end_idx <= len(new_loop):
            new_loop[start_idx:end_idx] += sample[:len(new_loop) - start_idx]  # trim if necessary

# Normalize to avoid clipping, then convert to int16
new_loop /= np.max(np.abs(new_loop)) + 1e-7
new_loop_int16 = np.int16(new_loop * 32767)

wavf.write("loop_out.wav", sr, new_loop_int16)


123
[1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0.]


In [ ]:
print(type(new_loop))

<class 'numpy.ndarray'>


In [292]:
new_loop = (new_loop * (2 ** 15 - 1)).astype("<h")

with wave.open("sound1.wav", "w") as f:
    # 2 Channels.
    f.setnchannels(1)
    # 2 bytes per sample.
    f.setsampwidth(2)
    f.setframerate(sr)
    f.writeframes(new_loop.tobytes())

In [ ]:
import scipy.io.wavfile as wavf

out_f = 'loop_out.wav'

wavf.write(out_f, sr, new_loop)

In [193]:
print(1/60)

0.016666666666666666
